In [19]:
#MNIST Reservoir                         #07/16/2021

import reservoirpy as rpy

from reservoirpy.nodes import Reservoir, Ridge

#Create Reservoir
reservoir = Reservoir(100, lr=0.5, sr=0.9)
#Create Readout Layer
readout_layer = Ridge(ridge=1e-7) #Ridge is the readout lyaer: uses linear regression

MNIST_ESN = reservoir >> readout_layer #This connects the readout_layer node to the reservoir


In [20]:
import numpy
import matplotlib.pyplot

#Format Mnist Training Data: first value in the set is the Lable, following values are pixel values 

tr_data_file = open(r"/Users/niharkoppolu/Desktop/Number_Detection_Project/mnist_data_set/mnist_train.csv")
tr_data_list = tr_data_file.readlines()
tr_data_file.close()

#The following code is USED AS A Test: the first value in the MNIST list is broken up into the Label and the pixels.
#Then the pixels are printed in a grid to produce a visual of the number
"""
set1_values = tr_data_list[0].split(',')
#print label
label = set1_values[0]
print("label: ", label)

image_array = numpy.asfarray(set1_values[1:]).reshape((28,28))
matplotlib.pyplot.imshow(image_array, cmap='Greys', interpolation='None')
"""
#tr_len = len(tr_data_list)
tr_len = 5000
breaker = 0

index = 0
all_inputs = numpy.empty((tr_len, 784))
all_targets = numpy.empty((tr_len,10))
for MNIST_single_value in tr_data_list:
#used to end LOOP
    if breaker >= tr_len: 
        break;

    #convert test data from list off comma sep. vals to array of floats
    single_value_list = MNIST_single_value.split(',')

    #rescale data from 0 - 255 to range btw 0 - 0.99
    all_inputs[index] = (numpy.asfarray(single_value_list[1:]) / 255.0 * 0.99) + 0.01

    no_targets = len(all_targets)
    possible_range = 10
    for i in range(no_targets):
        for index in range(possible_range):
            #sets all values in array associated with target to 0.01
            all_targets[i][index] = 0.01
        #sets the array index associated with the correct value to 0.99
        all_targets[i][int(single_value_list[0])] = 0.99
    
    
    index += 1
    #used to end loop
    breaker += 1
    pass

In [21]:
#Training the Reservoir

MNIST_ESN = MNIST_ESN.fit(all_inputs, all_targets, warmup=0) #This trains the esn_model using the X_train as the input data and Y_train as the target data 
# Does the same thing as the previous program, just more condensed

print(reservoir.is_initialized, readout_layer.is_initialized, readout_layer.fitted)

Running Model-3: 5000it [00:00, 19515.02it/s]?, ?it/s]
Running Model-3: 100%|██████████| 1/1 [00:00<00:00,  3.75it/s]


Fitting node Ridge-3...
True True True


In [22]:
#read in TEST data
test_data_file = open(r"/Users/niharkoppolu/Desktop/Number_Detection_Project/mnist_data_set/mnist_test.csv")
test_data_list = test_data_file.readlines()
test_data_file.close()

no_of_tests = 0

all_test_pixels = numpy.empty((10, 784))
all_correct_values = numpy.empty((10,10))

while no_of_tests != 10:

    single_test = test_data_list[no_of_tests].split(',')
    all_test_pixels[no_of_tests] = (numpy.asfarray(single_test[1:]) / 255.0 * 0.99) + 0.01 

    for i in range(10):
        #sets all values in array associated with target to 0.01
        all_correct_values[no_of_tests][i] = 0.01
    
    all_correct_values[no_of_tests][int(single_test[0])] = 0.99
    
    no_of_tests += 1
    pass
    

In [23]:
def Find_Correct_Index(array):
     highest_val = max(array)
     
     for correct_index in range(10):
         if array[correct_index] == highest_val:
             return correct_index
        

In [24]:
#Run Tests

Predicted_val_set = numpy.empty((10,10))

i = 0 
no_correct = 0
while i != 10:

    Predicted_val_set[i] = MNIST_ESN.run(all_test_pixels[i])
    Predicted_Answer = Find_Correct_Index(Predicted_val_set[i])
    Correct_Answer = Find_Correct_Index(all_correct_values[i])

    print("Predicted value: ", Predicted_Answer)
    print("Correct value: ", Correct_Answer)
    if int(Predicted_Answer) == Correct_Answer: 
        no_correct += 1
        pass

    i += 1
    pass

percent_correct = (no_correct / i) * 100

print("Performance of Reservoir: ", percent_correct)

Running Model-3: 100%|██████████| 1/1 [00:00<00:00, 434.42it/s]


Predicted value:  2
Correct value:  7


Running Model-3: 100%|██████████| 1/1 [00:00<00:00, 3894.43it/s]


Predicted value:  2
Correct value:  2


Running Model-3: 100%|██████████| 1/1 [00:00<00:00, 4975.45it/s]


...

Predicted value:  2
Correct value:  9


Running Model-3: 100%|██████████| 1/1 [00:00<00:00, 2993.79it/s]


Predicted value:  2
Correct value:  5


Running Model-3: 100%|██████████| 1/1 [00:00<00:00, 6374.32it/s]

Predicted value:  2
Correct value:  9
The number of correct predictions done by the reservoir is:  10.0
